In [15]:
import numpy as np
import scipy.io as sio


 '''This method loads the training, validation and test set.
    It also divides the training set into mini-batches.
    Inputs:
        N: Mini-batch size.
    Outputs:
        train_input: An array of size D X N X M, where
            D: number of input dimensions (in this case, 3).
            N: size of each mini-batch (in this case, 100).
            M: number of minibatches.
        train_target: An array of size 1 X N X M.
        valid_input: An array of size D X number of points in the validation set.
        test: An array of size D X number of points in the test set.
        vocab: Vocabulary containing index to word mapping.
    '''

In [17]:
N=1
data = sio.loadmat('data.mat')
trainData = data['data'][0][0]['trainData']
validData = data['data'][0][0]['validData']
testData = data['data'][0][0]['testData']
vocab = data['data'][0][0]['vocab']

numdims = trainData.shape[0]
D = numdims - 1
M = int(trainData.shape[1] / N)

train_input = trainData[0:D, 0:N * M].reshape(D, N, M, order="F")
train_target = trainData[D, 0:N * M].reshape(1, N, M, order="F")

valid_input = validData[0:D]
valid_target = validData[D].reshape(1, validData.shape[1], order="F")
test_input = testData[0:D]
test_target = testData[D].reshape(1, testData.shape[1], order="F")


In [18]:
assert trainData.shape == (4, 372550)
assert validData.shape == (4, 46568)

In [19]:
train_input.shape

(3, 1, 372550)

In [21]:
batchsize = 100  # Mini-batch size.
learning_rate = 0.1  # Learning rate; default = 0.1.
momentum = 0.9  # Momentum; default = 0.9.
numhid1 = 50  # Dimensionality of embedding space; default = 50.
numhid2 = 200  # Number of units in hidden layer; default = 200.
init_wt = 0.01  # Standard deviation of the normal distribution
# which is sampled to get the initial weights; default = 0.01

# VARIABLES FOR TRACKING TRAINING PROGRESS
show_training_CE_after = 100
show_validation_CE_after = 1000

In [22]:
# Decrease all word indices by 1: 1-250 -> 0-249
train_input = train_input - 1
train_target = train_target - 1
valid_input = valid_input - 1
valid_target = valid_target - 1
test_input = test_input - 1
test_target = test_target - 1


In [23]:
[numwords, batchsize, numbatches] = train_input.shape
vocab_size = vocab.shape[1]

In [24]:
# INITIALIZE WEIGHTS AND BIASES
word_embedding_weights = init_wt * np.random.randn(vocab_size, numhid1)
embed_to_hid_weights = init_wt * np.random.randn(numwords * numhid1, numhid2)
hid_to_output_weights = init_wt * np.random.randn(numhid2, vocab_size)
hid_bias = np.zeros((numhid2, 1))
output_bias = np.zeros((vocab_size, 1))

In [27]:
word_embedding_weights_delta = np.zeros((vocab_size, numhid1))
word_embedding_weights_gradient = np.zeros((vocab_size, numhid1))
embed_to_hid_weights_delta = np.zeros((numwords * numhid1, numhid2))
hid_to_output_weights_delta = np.zeros((numhid2, vocab_size))
hid_bias_delta = np.zeros((numhid2, 1))
output_bias_delta = np.zeros((vocab_size, 1))
expansion_matrix = np.eye(vocab_size)
count = 0
tiny = np.exp(-30)

In [28]:
epoch=1